In [1]:
!pip install qiskit
import numpy as np
from math import log2, sqrt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00


In [2]:
# Define quantum gates
H = 1/sqrt(2) * np.array([[1, 1], [1, -1]])  # Hadamard gate
I = np.eye(2)                                # Identity gate
CNOT = np.array([[1,0,0,0], [0,1,0,0], [0,0,0,1], [0,0,1,0]])  # CNOT gate

In [3]:
class BellStates:
    @staticmethod
    def phi_plus():
        """Construct |Φ⁺⟩ = (|00⟩ + |11⟩)/√2"""
        state = np.kron([1, 0], [1, 0])  # |00⟩
        state = np.kron(H, I) @ state    # Apply H to first qubit
        return CNOT @ state              # Apply CNOT

    @staticmethod
    def phi_minus():
        """Construct |Φ⁻⟩ = (|00⟩ - |11⟩)/√2"""
        state = np.kron([0, 1], [1, 0])  # |10⟩
        state = np.kron(H, I) @ state
        return CNOT @ state

    @staticmethod
    def psi_plus():
        """Construct |Ψ⁺⟩ = (|01⟩ + |10⟩)/√2"""
        state = np.kron([1, 0], [0, 1])  # |01⟩
        state = np.kron(H, I) @ state
        return CNOT @ state

    @staticmethod
    def psi_minus():
        """Construct |Ψ⁻⟩ = (|01⟩ - |10⟩)/√2"""
        state = np.kron([0, 1], [0, 1])  # |11⟩
        state = np.kron(H, I) @ state
        return CNOT @ state

In [4]:
def partial_trace(rho, dims, axis=0):
    """
    Compute partial trace of density matrix rho
    dims: list of dimensions of each subsystem [dA, dB]
    axis: 0 for tracing out B, 1 for tracing out A
    """
    dA, dB = dims
    if axis == 0:  # Trace out B
        rho_reduced = np.zeros((dA, dA), dtype=complex)
        for i in range(dA):
            for j in range(dA):
                for k in range(dB):
                    rho_reduced[i,j] += rho[i*dB + k, j*dB + k]
    else:  # Trace out A
        rho_reduced = np.zeros((dB, dB), dtype=complex)
        for i in range(dB):
            for j in range(dB):
                for k in range(dA):
                    rho_reduced[i,j] += rho[k*dB + i, k*dB + j]
    return rho_reduced


In [5]:
def entanglement_entropy(state):
    """
    Calculate entanglement entropy of bipartite state
    Input: state vector or density matrix
    Output: entanglement entropy
    """
    # Convert state to density matrix if it's a state vector
    if state.ndim == 1:
        rho = np.outer(state, state.conj())
    else:
        rho = state

     # Partial trace over subsystem B (assuming 2-qubit system)
    rho_A = partial_trace(rho, [2, 2], axis=1)

    # Compute eigenvalues (using eigh for Hermitian matrices)
    eigvals = np.linalg.eigvalsh(rho_A)

    # Calculate von Neumann entropy
    entropy = 0.0
    for lamda in eigvals:
        if lamda > 1e-10:  # avoid log(0)
            entropy -= lamda * log2(lamda)

    return entropy

In [6]:
   # Example usage
if __name__ == "__main__":
    # Construct Bell states
    phi_p = BellStates.phi_plus()
    phi_m = BellStates.phi_minus()
    psi_p = BellStates.psi_plus()
    psi_m = BellStates.psi_minus()

    print("\n" + "="*50)
    print("TASK 3: BELL STATES AND ENTANGLEMENT ENTROPY")
    print("="*50)

    print(f"Bell state |Φ⁺⟩ =", phi_p)
    print(f"Bell state |Φ⁻⟩ =", phi_m)
    print(f"Bell state |Ψ⁺⟩ =", psi_p)
    print(f"Bell state |Ψ⁻⟩ =", psi_m)

   # Verify entanglement entropy (should be 1 for maximally entangled states)
    print(f"Entanglement entropy of |Φ⁺⟩: {entanglement_entropy(phi_p):.4f}")
    print(f"Entanglement entropy of |Φ⁻⟩: {entanglement_entropy(phi_m):.4f}")
    print(f"Entanglement entropy of |Ψ⁺⟩: {entanglement_entropy(psi_p):.4f}")
    print(f"Entanglement entropy of |Ψ⁻⟩: {entanglement_entropy(psi_m):.4f}")

    # Verify product state has zero entanglement entropy
    product_state = np.kron([1, 0], [1, 0])  # |00⟩
    print(f"Entanglement entropy of |00⟩: {entanglement_entropy(product_state):.4f}")



TASK 3: BELL STATES AND ENTANGLEMENT ENTROPY
Bell state |Φ⁺⟩ = [0.70710678 0.         0.         0.70710678]
Bell state |Φ⁻⟩ = [ 0.70710678  0.          0.         -0.70710678]
Bell state |Ψ⁺⟩ = [0.         0.70710678 0.70710678 0.        ]
Bell state |Ψ⁻⟩ = [ 0.          0.70710678 -0.70710678  0.        ]
Entanglement entropy of |Φ⁺⟩: 1.0000
Entanglement entropy of |Φ⁻⟩: 1.0000
Entanglement entropy of |Ψ⁺⟩: 1.0000
Entanglement entropy of |Ψ⁻⟩: 1.0000
Entanglement entropy of |00⟩: 0.0000


In [12]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace, entropy
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd

# Initialize simulator
simulator = AerSimulator()

def create_bell_state(bell_type):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    if bell_type == 'phi_minus':
        qc.z(0)
    elif bell_type == 'psi_plus':
        qc.x(1)
    elif bell_type == 'psi_minus':
        qc.z(0)
        qc.x(1)
    return qc

def calculate_entropy(qc):
    statevector = Statevector.from_instruction(qc)
    rho = partial_trace(statevector, [1])
    return entropy(rho, base=2)

state_data = []
states = {
    'phi_plus': '|Φ⁺⟩',
    'phi_minus': '|Φ⁻⟩',
    'psi_plus': '|Ψ⁺⟩',
    'psi_minus': '|Ψ⁻⟩',
    'product': '|00⟩'
}

print("Bell States Construction and Entanglement Analysis")
print("="*50)

for key, label in states.items():
    if key == 'product':
        qc = QuantumCircuit(2)
        print("\nProduct State:")
    else:
        qc = create_bell_state(key)
        print(f"\nBell State {label}:")

    # Display circuit
    print("Quantum Circuit:")
    print(qc)

    # Calculate and display results
    entropy_val = calculate_entropy(qc)
    statevector = Statevector.from_instruction(qc)

    print("\nStatevector:")
    print(np.round(statevector.data, 3))

    print(f"Entanglement Entropy: {round(entropy_val, 3)}")
    print("-"*40)

    state_data.append({
        'State': label,
        'Entanglement Entropy': round(entropy_val, 3),
        'Score (out of 1)': f"{round(entropy_val, 3)}/1"
    })

# Create and display summary table
print("\nSummary Table:")
print("="*50)
df = pd.DataFrame(state_data)
print(df.to_string(index=False))

Bell States Construction and Entanglement Analysis

Bell State |Φ⁺⟩:
Quantum Circuit:
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

Statevector:
[0.707+0.j 0.   +0.j 0.   +0.j 0.707+0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Φ⁻⟩:
Quantum Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘     

Statevector:
[ 0.707+0.j  0.   +0.j  0.   +0.j -0.707+0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Ψ⁺⟩:
Quantum Circuit:
     ┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐┌───┐
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

Statevector:
[0.   +0.j 0.707+0.j 0.707+0.j 0.   +0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Ψ⁻⟩:
Quantum Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐├───┤
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

Statevector:
[ 0.   +0.j -0.707+0.j  0.7

In [13]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace, entropy
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd

# Initialize simulator
simulator = AerSimulator()

def create_bell_state(bell_type):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    if bell_type == 'phi_minus':
        qc.z(0)
    elif bell_type == 'psi_plus':
        qc.x(1)
    elif bell_type == 'psi_minus':
        qc.z(0)
        qc.x(1)
    return qc

def calculate_entropy(qc):
    statevector = Statevector.from_instruction(qc)
    rho = partial_trace(statevector, [1])
    return entropy(rho, base=2)

state_data = []
states = {
    'phi_plus': '|Φ⁺⟩',
    'phi_minus': '|Φ⁻⟩',
    'psi_plus': '|Ψ⁺⟩',
    'psi_minus': '|Ψ⁻⟩',
    'product': '|00⟩'
}

print("Bell States Construction and Entanglement Analysis")
print("="*50)

for key, label in states.items():
    if key == 'product':
        qc = QuantumCircuit(2)
        print("\nProduct State:")
    else:
        qc = create_bell_state(key)
        print(f"\nBell State {label}:")

    # Display circuit
    print("Quantum Circuit:")
    print(qc)

    # Calculate and display results
    entropy_val = calculate_entropy(qc)
    statevector = Statevector.from_instruction(qc)

    print("\nStatevector:")
    print(np.round(statevector.data, 3))

    print(f"Entanglement Entropy: {round(entropy_val, 3)}")
    print("-"*40)

    state_data.append({
        'State': label,
        'Entanglement Entropy': round(entropy_val, 3),
        'Score (out of 1)': f"{round(entropy_val, 3)}/1"
    })

# Create and display summary table
print("\nSummary Table:")
print("="*50)
df = pd.DataFrame(state_data)
print(df.to_string(index=False))

Bell States Construction and Entanglement Analysis

Bell State |Φ⁺⟩:
Quantum Circuit:
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

Statevector:
[0.707+0.j 0.   +0.j 0.   +0.j 0.707+0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Φ⁻⟩:
Quantum Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘     

Statevector:
[ 0.707+0.j  0.   +0.j  0.   +0.j -0.707+0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Ψ⁺⟩:
Quantum Circuit:
     ┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐┌───┐
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

Statevector:
[0.   +0.j 0.707+0.j 0.707+0.j 0.   +0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Ψ⁻⟩:
Quantum Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐├───┤
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

Statevector:
[ 0.   +0.j -0.707+0.j  0.7

In [9]:
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 95.6 MB/s eta 0:00:00


In [10]:
!pip install qiskit-aer

In [11]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace, entropy
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd

# Initialize simulator
simulator = AerSimulator()

def create_bell_state(bell_type):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    if bell_type == 'phi_minus':
        qc.z(0)
    elif bell_type == 'psi_plus':
        qc.x(1)
    elif bell_type == 'psi_minus':
        qc.z(0)
        qc.x(1)
    return qc

def calculate_entropy(qc):
    statevector = Statevector.from_instruction(qc)
    rho = partial_trace(statevector, [1])
    return entropy(rho, base=2)

state_data = []
states = {
    'phi_plus': '|Φ⁺⟩',
    'phi_minus': '|Φ⁻⟩',
    'psi_plus': '|Ψ⁺⟩',
    'psi_minus': '|Ψ⁻⟩',
    'product': '|00⟩'
}

print("Bell States Construction and Entanglement Analysis")
print("="*50)

for key, label in states.items():
    if key == 'product':
        qc = QuantumCircuit(2)
        print("\nProduct State:")
    else:
        qc = create_bell_state(key)
        print(f"\nBell State {label}:")

    # Display circuit
    print("Quantum Circuit:")
    print(qc)

    # Calculate and display results
    entropy_val = calculate_entropy(qc)
    statevector = Statevector.from_instruction(qc)

    print("\nStatevector:")
    print(np.round(statevector.data, 3))

    print(f"Entanglement Entropy: {round(entropy_val, 3)}")
    print("-"*40)

    state_data.append({
        'State': label,
        'Entanglement Entropy': round(entropy_val, 3),
        'Score (out of 1)': f"{round(entropy_val, 3)}/1"
    })

# Create and display summary table
print("\nSummary Table:")
print("="*50)
df = pd.DataFrame(state_data)
print(df.to_string(index=False))

Bell States Construction and Entanglement Analysis

Bell State |Φ⁺⟩:
Quantum Circuit:
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

Statevector:
[0.707+0.j 0.   +0.j 0.   +0.j 0.707+0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Φ⁻⟩:
Quantum Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘     

Statevector:
[ 0.707+0.j  0.   +0.j  0.   +0.j -0.707+0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Ψ⁺⟩:
Quantum Circuit:
     ┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐┌───┐
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

Statevector:
[0.   +0.j 0.707+0.j 0.707+0.j 0.   +0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Ψ⁻⟩:
Quantum Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐├───┤
q_1: ─────┤ X ├┤ X ├
          └───┘└───┘

Statevector:
[ 0.   +0.j -0.707+0.j  0.7

In [14]:
import numpy as np
from numpy.linalg import eig

print("\n" + "="*50)
print("TASK 2: PAULI MATRICES AND EIGEN-ANALYSIS")
print("="*50)

# Define Pauli matrices
pauli_x = np.array([[0, 1], [1, 0]])
pauli_y = np.array([[0, -1j], [1j, 0]])
pauli_z = np.array([[1, 0], [0, -1]])

print("Pauli-X matrix:")
print(pauli_x)
print("\nPauli-Y matrix:")
print(pauli_y)
print("\nPauli-Z matrix:")
print(pauli_z)

# Apply to qubit states
qubit_0 = np.array([1, 0])  # |0⟩
qubit_1 = np.array([0, 1])  # |1⟩
print("\nApplying Pauli-X to |0⟩:", pauli_x @ qubit_0)
print("Applying Pauli-X to |1⟩:", pauli_x @ qubit_1)

# Compute eigenvalues and eigenvectors
def analyze_operator(matrix, name):
    eigenvals, eigenvecs = eig(matrix)
    print(f"\n{name} Eigenvalues:", eigenvals)
    print(f"{name} Eigenvectors:")
    for i, vec in enumerate(eigenvecs.T):
        print(f" λ={eigenvals[i]:.1f}: {vec}")

analyze_operator(pauli_x, "Pauli-X")
analyze_operator(pauli_y, "Pauli-Y")
analyze_operator(pauli_z, "Pauli-Z")



TASK 2: PAULI MATRICES AND EIGEN-ANALYSIS
Pauli-X matrix:
[[0 1]
 [1 0]]

Pauli-Y matrix:
[[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]

Pauli-Z matrix:
[[ 1  0]
 [ 0 -1]]

Applying Pauli-X to |0⟩: [0 1]
Applying Pauli-X to |1⟩: [1 0]

Pauli-X Eigenvalues: [ 1. -1.]
Pauli-X Eigenvectors:
 λ=1.0: [0.70710678 0.70710678]
 λ=-1.0: [-0.70710678  0.70710678]

Pauli-Y Eigenvalues: [ 1.+0.j -1.+0.j]
Pauli-Y Eigenvectors:
 λ=1.0+0.0j: [-0.        -0.70710678j  0.70710678+0.j        ]
 λ=-1.0+0.0j: [0.70710678+0.j         0.        -0.70710678j]

Pauli-Z Eigenvalues: [ 1. -1.]
Pauli-Z Eigenvectors:
 λ=1.0: [1. 0.]
 λ=-1.0: [0. 1.]


In [15]:
import numpy as np
from math import log2, sqrt

# Define quantum gates
H = 1 / sqrt(2) * np.array([[1, 1], [1, -1]])  # Hadamard gate
I = np.eye(2)                                   # Identity gate
CNOT = np.array([[1,0,0,0], [0,1,0,0], [0,0,0,1], [0,0,1,0]])  # CNOT gate

class BellStates:
    @staticmethod
    def phi_plus():
        """Construct |Φ⁺⟩ = (|00⟩ + |11⟩)/√2"""
        state = np.kron([1, 0], [1, 0])  # |00⟩
        state = np.kron(H, I) @ state    # Apply H to first qubit
        return CNOT @ state              # Apply CNOT

    @staticmethod
    def phi_minus():
        """Construct |Φ⁻⟩ = (|00⟩ - |11⟩)/√2"""
        # Start from |00⟩ but apply Z on first qubit after entangling
        state = BellStates.phi_plus()
        # Apply Z on first qubit: Z = [[1,0],[0,-1]]
        Z = np.array([[1, 0], [0, -1]])
        Z1 = np.kron(Z, I)
        return Z1 @ state

    @staticmethod
    def psi_plus():
        """Construct |Ψ⁺⟩ = (|01⟩ + |10⟩)/√2"""
        # Start from |Φ⁺⟩ and apply X on second qubit
        state = BellStates.phi_plus()
        X = np.array([[0, 1], [1, 0]])
        X2 = np.kron(I, X)
        return X2 @ state

    @staticmethod
    def psi_minus():
        """Construct |Ψ⁻⟩ = (|01⟩ - |10⟩)/√2"""
        # Start from |Φ⁻⟩ and apply X on second qubit
        state = BellStates.phi_minus()
        X = np.array([[0, 1], [1, 0]])
        X2 = np.kron(I, X)
        return X2 @ state

def partial_trace(rho, dims, axis=0):
    """
    Compute partial trace of density matrix rho
    dims: list of dimensions of each subsystem [dA, dB]
    axis: 0 for tracing out B, 1 for tracing out A
    """
    dA, dB = dims
    if axis == 0:  # Trace out B
        rho_reduced = np.zeros((dA, dA), dtype=complex)
        for i in range(dA):
            for j in range(dA):
                for k in range(dB):
                    rho_reduced[i,j] += rho[i*dB + k, j*dB + k]
    else:  # Trace out A
        rho_reduced = np.zeros((dB, dB), dtype=complex)
        for i in range(dB):
            for j in range(dB):
                for k in range(dA):
                    rho_reduced[i,j] += rho[k*dB + i, k*dB + j]
    return rho_reduced

def entanglement_entropy(state):
    """
    Calculate entanglement entropy of bipartite state
    Input: state vector or density matrix
    Output: entanglement entropy
    """


    if state.ndim == 1:
        rho = np.outer(state, state.conj())
    else:
        rho = state


    rho_A = partial_trace(rho, [2, 2], axis=1)


    eigvals = np.linalg.eigvalsh(rho_A)


    entropy = 0.0
    for lamda in eigvals:
        if lamda > 1e-12:  # avoid log(0)
            entropy -= lamda * log2(lamda)

    return entropy

if __name__ == "__main__":

    phi_p = BellStates.phi_plus()
    phi_m = BellStates.phi_minus()
    psi_p = BellStates.psi_plus()
    psi_m = BellStates.psi_minus()

    print("\n" + "="*50)
    print("TASK: BELL STATES AND ENTANGLEMENT ENTROPY (No Qiskit)")
    print("="*50)

    print(f"Bell state |Φ⁺⟩ = {np.round(phi_p, 3)}")
    print(f"Bell state |Φ⁻⟩ = {np.round(phi_m, 3)}")
    print(f"Bell state |Ψ⁺⟩ = {np.round(psi_p, 3)}")
    print(f"Bell state |Ψ⁻⟩ = {np.round(psi_m, 3)}")


    print(f"Entanglement entropy of |Φ⁺⟩: {entanglement_entropy(phi_p):.4f}")
    print(f"Entanglement entropy of |Φ⁻⟩: {entanglement_entropy(phi_m):.4f}")
    print(f"Entanglement entropy of |Ψ⁺⟩: {entanglement_entropy(psi_p):.4f}")
    print(f"Entanglement entropy of |Ψ⁻⟩: {entanglement_entropy(psi_m):.4f}")


    product_state = np.kron([1, 0], [1, 0])  # |00⟩
    print(f"Entanglement entropy of |00⟩: {entanglement_entropy(product_state):.4f}")



TASK: BELL STATES AND ENTANGLEMENT ENTROPY (No Qiskit)
Bell state |Φ⁺⟩ = [0.707 0.    0.    0.707]
Bell state |Φ⁻⟩ = [ 0.707  0.     0.    -0.707]
Bell state |Ψ⁺⟩ = [0.    0.707 0.707 0.   ]
Bell state |Ψ⁻⟩ = [ 0.     0.707 -0.707  0.   ]
Entanglement entropy of |Φ⁺⟩: 1.0000
Entanglement entropy of |Φ⁻⟩: 1.0000
Entanglement entropy of |Ψ⁺⟩: 1.0000
Entanglement entropy of |Ψ⁻⟩: 1.0000
Entanglement entropy of |00⟩: 0.0000
